# LangChain: Memory
[As part of LangChain for LLM Application Development course by Deeplearning.ai]

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

Large Language Models are stateless and each transaction is independent. However, in order to have coherent conversations, chatbots appear to have memory providing the full conversation as 'context'. LangChain provides different types of memory that provide different ways to store the conversation context.

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain

## ConversationBufferMemory

In [30]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [31]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

Creating a chatbot using using a LangChain. The following cell demonstrates steps to create a ConversationChain. The verbose=True provides additional details on working of LangChain in the backend to generate a response for the user.

In [34]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True)

In [35]:
conversation.predict(input="Hi, my name is Maitreyee")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Maitreyee
AI:

> Finished chain.


"Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?"

In [36]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Maitreyee
AI: Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [37]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Maitreyee
AI: Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Maitreyee, as you mentioned earlier.'

The Current Conversation section displayed by verbose=True displays LangChain storing previous conversation details in the ConversationBufferMemory and hence, it can respond to the question 'What is your name' with 'Andrew' from the information stored in the memory buffer.

In [38]:
print(memory.buffer)

Human: Hi, my name is Maitreyee
AI: Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Maitreyee, as you mentioned earlier.


The load_memory_variables() has empty dictionary input - {} that shows the format in which LangChain has stored the information from the chat.

In [39]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Maitreyee\nAI: Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Maitreyee, as you mentioned earlier."}

In [40]:
memory = ConversationBufferMemory()

User can add new information to the ConversationBufferMemory by using save_context() function with an dictionary input of information that needs to be added.

In [41]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [42]:
print(memory.buffer)

Human: Hi
AI: What's up


In [43]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [44]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [45]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

Langchain provides memory to store previous conversation for context. However, long conversation need more memory and that increases the cost of sending a lot of tokens to the LLM.

LangChain provides convenient types of memory to store and accumulate the conversation.

## ConversationBufferWindowMemory

ConversationBufferWindowMemory- This type of memory remembers a window of the past coversation. The window length can be specified in the arguments (k).

In [46]:
from langchain.memory import ConversationBufferWindowMemory

In [47]:
#k=1 makes the memory store the most recent conversation transaction.
memory = ConversationBufferWindowMemory(k=1)

In [48]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [49]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

The following example demonstrates ConversationChain with ConversationBufferWindowMemory set to 1. The memory is given to consecutive inputs Name followed by a addition prompt. However, since k=1, the Current Coversation in the memory only stores the latest addition chat transaction and does not store/remember the input name in the previous conversation transaction.

In [51]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [52]:
conversation.predict(input="Hi, my name is Maitreyee")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Maitreyee
AI:

> Finished chain.


"Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?"

In [53]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Maitreyee
AI: Hello Maitreyee, it's nice to meet you! I am an AI language model designed to assist with various tasks and answer questions. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [54]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory
This type of memory limits the storage on numbe rof tokens stored in the buffer. This influences the LLM pricing for the user.

In [55]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


The max_token_limit argument for the ConversationTokenBufferMemory() sets the number of recent tokens to be stored in the memory.

In [57]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [58]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

In [59]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

##ConversationSummaryMemory
This type of memory stores summary of the conversation.

In [60]:
from langchain.memory import ConversationSummaryBufferMemory

In [61]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [62]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [63]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

The current conversation now stores the summary of the conversation instead of the entire past conversation.

In [64]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."

In [65]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."}

Other kinds of memories supported by LangChain:

##Vector Data Memory
Stores text in a vector database and retrieves most relevant blocks of text.

##Entity Memory
Store details about specific entities using LLMs.

A combination of different type of memories can also be used.

A SQL or key-value database can be used to storage.
